In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import seaborn as sns

In [10]:
sns.set_context('poster', font_scale=1.25)

In [11]:
sns.set_style('ticks')

In [17]:
import expansion_interactions as ei

In [23]:
strain_names = ['mCherry', 'eCFP', 'Black', 'eYFP']
vw = [0, 0.09, 0.10, 0.14]

In [24]:
interactions = ei.Derive_From_Weakest_Strain(strain_names, vw, 'deterministic')

We now get the dataframe we want.

In [30]:
import pandas as pd

In [33]:
interact_list = []
for i in interactions.strain_interactions:
    # Only worry about positive interactions
    if i.s > 0:
        cur_list = []
        cur_list.append(i.strain_1.name)
        cur_list.append(i.strain_2.name)
        cur_list.append(i.s)
        cur_list.append(i.vw)
        cur_list.append(i.Ls)
        cur_list.append(i.kappa)
        interact_list.append(cur_list)
        
interact_df = pd.DataFrame(data=interact_list, columns=['Sweeper',
                                                        'Sweepee',
                                                        's',
                                                        'vw',
                                                        'Ls',
                                                        'kappa'])

In [35]:
interact_df.sort_values('Ls', inplace=True)

In [36]:
interact_df

,Sweeper,Sweepee,s,vw,Ls,kappa
3,eYFP,mCherry,0.070000,0.140000,4.250000,0.907485
4,eYFP,eCFP,0.041092,0.107265,7.239811,0.695297
1,Black,mCherry,0.035714,0.100000,8.330000,0.648204
5,eYFP,Black,0.034307,0.098010,8.671660,0.635306
0,eCFP,mCherry,0.028929,0.090000,10.283951,0.583383
2,Black,eCFP,0.006787,0.043595,43.830781,0.282582


Ok. Good. We need Dg in order to estimate whether the wave is noisy or not, however. Let's get that now.